In [1]:
# load packages
import os
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import scipy.stats as ss

ext_scripts_dir = ('/home/barbora/Documents/Projects/Normative_Models/ESO/braincharts/scripts')
os.chdir(ext_scripts_dir)

from nm_utils import remove_bad_subjects, load_2d

code_dir = ('/home/barbora/Documents/Projects/Normative_Models/ESO/code')
os.chdir(code_dir)

# importing custom functions
import clinics_desc_functions as custom

sns.set(rc={'figure.facecolor':'white'})

In [2]:
import prepare_models_for_analysis

model_name, site_names, site_ids_tr, idp_ids = custom.pretrained_ini()

# where things are
main_dir = ('/home/barbora/Documents/Projects/Normative_Models/COINS')
models_dir = ('/home/barbora/Documents/Projects/Normative_Models/COINS/models')
os.makedirs(models_dir, exist_ok=True)
fsdata_dir = ('/home/barbora/Documents/Projects/Normative_Models/COINS/backup')
pretrained_dir = ('/home/barbora/Documents/Projects/Normative_Models/ESO/braincharts')
images_dir = os.path.join(models_dir,'img')
os.makedirs(images_dir, exist_ok=True)


Load FS info
-------------

In [166]:
v1_fs = pd.read_csv(os.path.join(fsdata_dir,'fit_external_long_thickness_1.txt'), sep=';')
v1_fs.index = [i.split('.')[-1][2:] for i in v1_fs['id']]
v1_fs = v1_fs.drop(columns= 'id')
v1_fs.sort_index(inplace=True)

In [167]:
v2_fs = pd.read_csv(os.path.join(fsdata_dir,'fit_external_long_thickness_2.txt'), sep=';')
v2_fs.index = [i.split('.')[-1][2:] for i in v2_fs['id']]
v2_fs = v2_fs.drop(columns= 'id')
v2_fs.sort_index(inplace=True)

In [168]:
if sum(v1_fs.index == v2_fs.index) == v1_fs.shape[0]:
    print('Indicies acoss the two dataframes are in sync!')

Indicies acoss the two dataframes are in sync!


Load clinics
-----------

In [208]:
corr_clin = pd.read_csv(os.path.join(fsdata_dir, 'CoRR_AggregatedPhenotypicData.csv'))
pk = [str(i) for i in corr_clin['SUBID']]
corr_clin.index = pk
corr_clin.sort_index(inplace=True)

In [211]:
# delete subjects that were not processed
del_from_clin = list(set(corr_clin.index.unique()) - set(v1_fs.index))
corr_clin = corr_clin.drop(index= del_from_clin)
corr_clin = corr_clin[corr_clin['SESSION']=='Baseline']

In [212]:
# delete from v1_fs, v2_fs
del_from_fs = list(set(v1_fs.index) - set(corr_clin.index.unique()))
v1_fs = v1_fs.drop(index = del_from_fs)
v2_fs = v2_fs.drop(index = del_from_fs)

**Run Normative models**
-------------------------